In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_10_9_21,0.940314,0.938078,0.923628,0.933084,0.066054,0.097712,0.077747,0.088317,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
1,model_10_9_16,0.939030,0.938556,0.918081,0.931373,0.067476,0.096959,0.083393,0.090576,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
2,model_10_9_8,0.938866,0.942250,0.932686,0.939013,0.067657,0.091130,0.068525,0.080493,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
3,model_10_9_9,0.938584,0.942326,0.924097,0.935943,0.067970,0.091010,0.077269,0.084544,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
4,model_10_9_10,0.938561,0.941695,0.924031,0.935519,0.067995,0.092005,0.077337,0.085104,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
...,...,...,...,...,...,...,...,...,...,...
199,model_7_2_5,0.776001,0.798658,0.778307,0.797337,0.247901,0.095492,0.538120,0.303788,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
206,model_7_2_6,0.771840,0.696084,0.744844,0.754654,0.252506,0.144140,0.619348,0.367768,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
215,model_1_2_7,0.765882,0.956922,0.852108,0.921615,0.259099,0.076940,0.156436,0.114350,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
226,model_1_2_6,0.757390,0.956557,0.863219,0.925169,0.268498,0.077593,0.144684,0.109165,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
